In [57]:
import torch
import pandas as pd
from torch import nn

## Load network and norm data

In [58]:
# Load frequency model
fnn = torch.jit.load('net_fnadir.pt')
# Load vsg power model
pnn = torch.jit.load('net_Ppeak.pt')

In [59]:
# load norm dadta
fnorm = pd.read_csv('fnorm.csv')
pnorm = pd.read_csv('pnorm.csv')
norm = {'fnorm': fnorm, 'pnorm': pnorm }

In [60]:
fnorm

,M,D,Fg,Rg,fnadir
0,14.988180,7.492874,80.045126,124.959573,-0.013860
1,2.891501,1.438628,40.350784,43.546600,0.006387


In [61]:
pnorm

,M,D,Fg,Rg,Mvsg,Dvsg,Ppeak
0,14.988180,7.492874,80.045126,124.959573,4.018493,2.004602,0.143188
1,2.891501,1.438628,40.350784,43.546600,2.317657,1.151364,0.086313


## Test prediction results

### frequency prediction

In [62]:
fmean = fnorm.iloc[0].values
fstd = fnorm.iloc[1].values

In [63]:
xf = [12, 3, 100, 120]   
xf = (xf - fmean[0:4]) / fstd[0:4]

xf = torch.tensor(xf, dtype=torch.float32)
xf

tensor([-1.0334, -3.1230,  0.4945, -0.1139])

In [64]:
delta_Pe = 1

delta_f = fnn(xf).detach().numpy() * fstd[4] + fmean[4]

delta_f *= delta_Pe

delta_f

array([-0.01259668], dtype=float32)

### vsg power prediction

In [65]:
pmean = pnorm.iloc[0].values
pstd = pnorm.iloc[1].values

In [67]:
xp = [12, 3, 100, 120, 4, 2]

xp = (xp - pmean[0:6]) / pstd[0:6]

xp = torch.tensor(xp, dtype=torch.float32)

xp


tensor([-1.0334, -3.1230,  0.4945, -0.1139, -0.0080, -0.0040])

In [68]:
delta_Pvsg = pnn(xp).detach().numpy() * pstd[6] + pmean[6]

delta_Pvsg *= delta_Pe

delta_Pvsg

array([0.14504695], dtype=float32)

## Debug Note

TODO:

1. 确认andes的base，然后框定39case各个参数的优化空间

2. 在优化空间内，重新生成训练数据，并训练网络。把神经网络的训练预测结果和matlab仿真结果对比验证

3. 设计ieee 39 的cost，然后验证下dcopf的结果

4. 顺一遍 visopf fnadir的code，包括formulation + code

5. debug 优化结果
   和飞哥确认，gorubypy 能不能看哪些约束bounded